[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AI-HPC-Research-Team/AI-Basics/blob/main/10%20Visualization/visualization.ipynb)
# Molecules and Proteins: A Practical Notebook

This notebook is organised into three teaching modules:

1. **Proteins** – how to work with 3D protein structures, build contact maps, and use interactive 3D viewers.
2. **Small molecules** – how to represent molecules with SMILES, fingerprints and scaffolds, and how to explore their chemical space.
3. **Integrated assignment** – a mini workflow that combines single-cell RNA-seq preprocessing with basic molecular modelling and visualisation.


## Part 1 – Protein Structures and Contact Maps

In this part we will:

- Import standard scientific Python tools and structure-handling libraries.
- Load a protein structure from a PDB file.
- Extract Cα coordinates and compute a pairwise distance matrix.
- Convert the distance matrix into a binary **contact map** under a given cut-off.
- Plot the contact map and relate it back to the 3D structure (e.g. using nglview or similar tools).

The key idea is that a protein can be described both as a 3D object in space and as a 2D matrix of residue–residue contacts.

In [ ]:
!pip -qq install numpy matplotlib Biopython wget nglview "ipywidgets>=8.1.0" scaffoldgraph[rdkit,vis] chemplot py3Dmol scikit-learn scaffoldgraph adjustText  gdown
from google.colab import output
output.enable_custom_widget_manager()


In [ ]:
import Bio.PDB, numpy as np, matplotlib.pyplot as plt
import os, wget

os.makedirs("data", exist_ok=True)
os.makedirs("figure", exist_ok=True)
# download a PDB file from the RCSB PDB database
wget.download("https://files.rcsb.org/download/1UBQ.pdb", "./data/1ubq.pdb")
parser = Bio.PDB.PDBParser()

structure = parser.get_structure("1UBQ", "./data/1ubq.pdb")  # e.g., ubiquitin
# Extract Cα atom coordinates for all residues:
coords = [atom.get_coord() for atom in structure.get_atoms() if atom.get_name() == "CA"]
coords = np.array(coords)
dist_matrix = np.linalg.norm(coords[:, None, :] - coords[None, :, :], axis=-1)
contact_map = dist_matrix < 8.0  # boolean matrix of contacts under 8Å

plt.imshow(contact_map, cmap="Greys", origin="lower")
plt.title("Contact Map of 1UBQ (Ubiquitin)")
plt.savefig("./figure/contact_map.png")

In [ ]:
import nglview as nv

# Load a structure (by PDB ID or local file)
# view = nv.show_pdbid("1TUP")        # p53 core domain structure from PDB
view = nv.show_file("./data/1ubq.pdb")  # load local AlphaFold model for p53
view.representations = [{"type": "cartoon", "params": {"color": "residueindex"}}]
view

## Part 2 – Small-Molecule Representations and Chemical Space

Now we switch from protein structures to **small molecules**. In this part we will:

- Install and import **RDKit** and other chemistry tools.
- Convert SMILES strings to canonical forms.
- Compute simple fingerprints such as **MACCS keys**.
- Generate 2D drawings and basic 3D conformers.
- Visualise a small set of molecules in **chemical space** using dimensionality reduction.
- Take a first look at scaffolds and scaffold networks.

Again, try to connect the code to the concepts: molecular graphs, fingerprints, similarity, and the geometry of conformers.

In [ ]:
from rdkit import Chem

mol = Chem.MolFromSmiles("OC(=O)C1=CC=CC=C1OC(=O)C")
canonical = Chem.MolToSmiles(mol, canonical=True)
print(canonical)  # Output: CC(=O)Oc1ccccc1C(=O)O


In [ ]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys

# Read a molecule, e.g. benzene
smiles = 'c1ccccc1'
mol = Chem.MolFromSmiles(smiles)

# Compute MACCS keys
maccs_fp = MACCSkeys.GenMACCSKeys(mol)

# Output as a bit vector
print(maccs_fp)                # print directly
print(list(maccs_fp))          # convert to a list (vector of 0/1)
print(maccs_fp.GetNumBits())   # check length (typically 167 bits; bit 0 unused)


In [ ]:
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign

# Generate 3D conformer for aspirin
aspirin_smiles = "CC(=O)Oc1ccccc1C(=O)O"  # Aspirin SMILES
mol = Chem.MolFromSmiles(aspirin_smiles)
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)


# Generate image from 3D conformer
img_3d = Draw.MolToImage(mol, size=(300, 300), legend="Aspirin 3D (conformer)")
image_path_3d = "./figure/aspirin_3d.png"
img_3d.save(image_path_3d)
mol.GetConformer(0).GetPositions()

In [ ]:
img_3d

In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.decomposition import PCA
import numpy as np
from matplotlib import pyplot as plt

def fp2arr(fp):
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# Define molecules with names
molecules = {
    "Aspirin": "CC(=O)Oc1ccccc1C(=O)O",
    "Celecoxib": "CC1=CC=C(C=C1)C2=CC("+ \
        "=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F",
    "Benzene": "C1=CC=CC=C1",
    "Ibuprofen": "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",
    "Nicotine": "CN1CCC[C@H]1C2=CN=CC=C2",
    "Salbutamol": "CC(C)(C)NCC(C1=CC(=C(C=C1)O)CO)O"
}

# Compute fingerprints
fps = [AllChem.GetMorganFingerprintAsBitVect(
        Chem.MolFromSmiles(smiles), 2, 1024)
     for smiles in molecules.values()]
arr = np.array([fp2arr(fp) for fp in fps])

# Perform PCA
pca = PCA(n_components=2)
res = pca.fit_transform(arr)
pc = res.T

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(pc[0], pc[1], marker="o",
    color='skyblue', edgecolors='k')

# Annotate points
for i, name in enumerate(molecules.keys()):
    plt.text(pc[0][i] + 0.02, pc[1][i], name, fontsize=9)

cum_cr = sum(pca.explained_variance_ratio_)
plt.title("Chemical Space Map (PCA on" + \
    "Morgan Fingerprints)\n" + \
    "Cumulative Variance Explained = %.2f" % cum_cr)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.tight_layout()

img_path = "./figure/pca_plot_annotated.png"
plt.savefig(img_path)
plt.show()
img_path


In [ ]:
from chemplot import Plotter
from matplotlib import pyplot as plt

# Define molecules
molecules = {
    "Aspirin": "CC(=O)Oc1ccccc1C(=O)O",
    "Celecoxib": "CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F",
    "Benzene": "C1=CC=CC=C1",
    "Ibuprofen": "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",
    "Nicotine": "CN1CCC[C@H]1C2=CN=CC=C2",
    "Salbutamol": "CC(C)(C)NCC(C1=CC(=C(C=C1)O)CO)O"
}
smiles_list = list(molecules.values())
labels = list(molecules.keys())

# Generate Chemplot PCA map
cp = Plotter.from_smiles(smiles_list)
coords = cp.pca()

# Plot with annotations
plt.figure(figsize=(8, 6))
plt.scatter(coords.iloc[:, 0], coords.iloc[:, 1], color='skyblue', edgecolors='k')

# Add text labels
for i, name in enumerate(labels):
    plt.text(coords.iloc[i, 0] + 0.02, coords.iloc[i, 1], name, fontsize=9)

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Chemical Space Map (PCA via Chemplot)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import os
os.makedirs("./data", exist_ok=True)

import math
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import networkx as nx
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor

rdDepictor.SetPreferCoordGen(True)  # nicer 2D coordinates for depiction

# ---- 1) Collect each node's SMILES and a short label ----
smiles_by_node = {}
labels = {}
for n, d in network.nodes(data=True):
    smi = d.get("smiles")  # ScaffoldGraph usually stores 'smiles' on nodes
    # Fallback: sometimes the node key itself is a SMILES string
    if smi is None and isinstance(n, str):
        try:
            if Chem.MolFromSmiles(n) is not None:
                smi = n
        except Exception:
            pass
    smiles_by_node[n] = smi

    # Use SMILES (truncated) as the label; fall back to node id
    label = smi if smi else str(n)
    labels[n] = (label[:21] + "...") if len(label) > 24 else label

# ---- 2) Compute a layout with extra spacing (to avoid overlap) ----
n_nodes = max(1, network.number_of_nodes())
# Larger k => nodes pushed further apart. Base on graph size to keep spacing reasonable.
k = 100 / math.sqrt(n_nodes)
pos = nx.spring_layout(network, seed=42, k=k, iterations=200)

# Helpful extents for dynamic label offsets
xs = [p[0] for p in pos.values()]
ys = [p[1] for p in pos.values()]
x_span = (max(xs) - min(xs)) or 1.0
y_span = (max(ys) - min(ys)) or 1.0

# ---- 3) Draw edges first (under everything) ----
fig, ax = plt.subplots(figsize=(10, 10))
nx.draw_networkx_edges(
    network, pos,
    width=2, alpha=0.8, edge_color="#999999", ax=ax, arrows=False
)

# ---- 4) Render each node as a 2D molecule image (instead of a dot) ----
def mol_image_from_smiles(smi, size=(140, 140)):
    """Return a PIL image for the SMILES; None if invalid."""
    if not smi:
        return None
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    rdDepictor.Compute2DCoords(mol)
    return Draw.MolToImage(mol, size=size)

# Tune these two to balance node size vs. overlap
img_size = (140, 140)
img_zoom = 0.33  # smaller = less overlap risk

for n, (x, y) in pos.items():
    img = mol_image_from_smiles(smiles_by_node.get(n), size=img_size)
    if img is not None:
        im = OffsetImage(img, zoom=img_zoom)
        # frameon=False ensures no bounding box around the image
        ab = AnnotationBbox(im, (x, y), frameon=False, pad=0.0, bboxprops=dict(alpha=0), zorder=3)
        ax.add_artist(ab)
    else:
        # Fallback: small dot if depiction failed
        ax.plot(x, y, "o", markersize=4, color="#1f77b4", zorder=3)

# ---- 5) Add labels below each image (offset computed from layout span) ----
# Use a vertical offset in data units so text does not cover the molecule image
label_y_offset = 0.06 * y_span  # increase if you still see overlap

for n, (x, y) in pos.items():
    ax.text(
        x, y - label_y_offset,
        labels[n],
        fontsize=7, ha="center", va="top", family="monospace",
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.85),
        zorder=4,
        clip_on=False,  # allow label to extend outside axes if needed
    )

ax.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
import py3Dmol
import requests

# Download PDB file
pdb_id = '1OXR'
pdb_url = f'https://files.rcsb.org/download/{pdb_id}.pdb'
pdb_data = requests.get(pdb_url).text

# Visualization setup
view = py3Dmol.view(width=800, height=600)
view.addModel(pdb_data, 'pdb')

# Display protein chain A as cartoon
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'blue'}})

# Display ligand AIN as sticks
view.setStyle({'resn': 'AIN'}, {'stick': {'color': 'magenta'}})

# Display calcium ions (CA) as spheres
view.setStyle({'resn': 'CA'}, {'sphere': {'radius': 0.8, 'color': 'yellow'}})

# Optional: add a surface for the protein-ligand complex
view.addSurface(py3Dmol.VDW, {'opacity': 0.2})

# Zoom to the ligand region
view.zoomTo({'resn': 'AIN'})

# Render the viewer
view.show()


In [ ]:
import matplotlib.pyplot as plt
from adjustText import adjust_text
from rdkit import Chem
from chemplot import Plotter
import wget

data_url = "https://github.com/AI-HPC-Research-Team/AI-Basics/raw/refs/heads/main/10 Visualization/data/NSAIDs.smi?raw=1"
wget.download(data_url, "./data/NSAIDs.smi")

# ---- Load data ---------------------------------------------------------------
smiles_list = [s.split(" ")[0] for s in open('./data/NSAIDs.smi')]
labels      = [s.split("#")[1].strip() for s in open('./data/NSAIDs.smi')]

cp     = Plotter.from_smiles(smiles_list)
coords = cp.umap(random_state=500)                      # compute UMAP coordinates
clusters =  cp.cluster(n_clusters=10)["clusters"]

X = coords.to_numpy()                        # (n_samples, 2)

# ---------- Identify one representative per cluster ----------
rep_indices = []          # store row-index of chosen molecules
for cid in np.unique(clusters):
    idx      = np.where(clusters == cid)[0]      # indices of points in this cluster
    cluster_pts = X[idx]                         # coordinates of those points
    centroid  = cluster_pts.mean(axis=0)         # 2-D centroid
    # choose point with smallest Euclidean distance to centroid
    rep_idx   = idx[np.linalg.norm(cluster_pts - centroid, axis=1).argmin()]
    rep_indices.append(rep_idx)

# ---------------- Plot with cluster colours ----------------
fig, ax = plt.subplots(figsize=(10, 10))
scatter = ax.scatter(X[:, 0], X[:, 1],
                     c=clusters, cmap='tab10', edgecolors='k', s=40)

# highlight representatives
for i in rep_indices:
    ax.scatter(X[i, 0], X[i, 1], s=250,
               facecolors='none', edgecolors='red', lw=2)
    ax.text(X[i, 0], X[i, 1]+2, labels[i],
            fontsize=20, weight='bold', color='red', ha='center', va='center')

ax.set_xlabel('UMAP-1')
ax.set_ylabel('UMAP-2')
ax.set_title('UMAP Chemical Space with 10 Clusters and Representatives')
plt.grid(True, ls='--', alpha=0.3)
plt.tight_layout()
plt.show()